<h1>Table des matières<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Traitement-du-df-aircraft" data-toc-modified-id="Traitement-du-df-aircraft-1">Traitement du df aircraft</a></span></li><li><span><a href="#Traitement-du-df-airline" data-toc-modified-id="Traitement-du-df-airline-2">Traitement du df airline</a></span></li><li><span><a href="#Traitement-du-df-airport" data-toc-modified-id="Traitement-du-df-airport-3">Traitement du df airport</a></span></li><li><span><a href="#Traitement-du-df-country" data-toc-modified-id="Traitement-du-df-country-4">Traitement du df country</a></span></li><li><span><a href="#Traitement-du-df-city" data-toc-modified-id="Traitement-du-df-city-5">Traitement du df city</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import os
import json
import requests
from os import listdir
from os.path import join, isfile, isdir

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
%%html
<style>
.dataframe thead tr th {
    text-align: left;
}
.dataframe thead th {
    text-align: left;
}
</style>

In [ ]:
path_folder_data_api = "static_results"
path_folder_data_wiki = "data_scraping"

In [3]:
files = [f for f in listdir(path_folder_data_api) if isfile(join(path_folder_data_api, f))]
dic_columns = {}
all_df = []
for file in files:
    path_file = f"{path_folder_data_api}/{file}"
    with open(path_file, "r", encoding='utf-8') as f:
        items = json.load(f)
                
    data = pd.json_normalize(items)
    df = pd.DataFrame(data)
    
    all_df.append(df)
    cols = list(df.columns)
    dic_columns[file[:-5]]=cols
    
display(dic_columns)

{'aircraft': ['AircraftCode', 'AirlineEquipCode', 'NameOK'],
 'airlines': ['AirlineID', 'AirlineID_ICAO', 'NameOK'],
 'airports': ['AirportCode',
  'CityCode',
  'CountryCode',
  'LocationType',
  'UtcOffset',
  'TimeZoneId',
  'NameOK',
  'Position.Coordinate.Latitude',
  'Position.Coordinate.Longitude'],
 'cities': ['CityCode',
  'CountryCode',
  'UtcOffset',
  'TimeZoneId',
  'NameOK',
  'Airports.AirportCode'],
 'countries': ['CountryCode', 'NameOK']}

In [4]:
dic_columns_new = {
    'aircraft': [
        'aircraft_iata',
        'aircraft_icao',
        'aircraft_name'
    ],
    'airlines': [
        'airline_iata',
        'airline_icao',
        'airline_name'
    ],
    'airports': [
        'airport_iata',
        'airport_city_code',
        'airport_country_code',
        'airport_location_type',
        'airport_utc_offset',
        'airport_timezone_id',
        'airport_name',
        'airport_latitude',
        'airport_longitude'
    ],
    'cities': [
        'city_code',
        'city_country_code',
        'city_utc_offset',
        'city_timezone_id',
        'city_name',
        'city_airport_code'
    ],
    'countries': [
        'country_code', 
        'country_name'
    ]
}

In [5]:
aircraft = all_df[0]
airline = all_df[1]
airport = all_df[2]
city = all_df[3]
country = all_df[4]

df_data = [
    aircraft,
    airline,
    airport,
    city,
    country
]

for i in range(len(df_data)):
    df_data[i].columns = dic_columns_new[files[i][:-5]]

In [6]:
def get_nan_from_df(df):
    cols = df.loc[:, df.isna().any()].columns
    dic_result = {}
    dic_result['df_nan'] = df[df.isna().any(axis = 1)]
    dic_result['col_nan'] = []
    for col in cols:
        indices_nan = df[[col]][df[[col]].isna().any(axis = 1)].index
        dic = {
            "col_name": col,
            "nb_nan": len(df.loc[df[col].isna()]),
            #"df": df[[col]][df[[col]].isna().any(axis = 1)]
            "df": df[df.index.isin(indices_nan)]
        }
        dic_result['col_nan'].append(dic)
    return dic_result

In [7]:
def display_nan(df):
    df_nan = get_nan_from_df(df)
    print("Synthèse :")
    dic_summary = {dic['col_name']: dic['nb_nan'] for dic in df_nan['col_nan']}
    display(dic_summary)
    display(df_nan['df_nan'])
    for dic in df_nan['col_nan']:
        print(f'Nb de NaN dans la colonne "{dic["col_name"]}" : {dic["nb_nan"]}')
        display(dic['df'])

## Traitement du df aircraft

In [8]:
# affichage du df
aircraft = all_df[0]
print(aircraft.shape)
display(aircraft.sample(10))

(382, 3)


,aircraft_iata,aircraft_icao,aircraft_name
113,77X,B772,Boeing 777-200F Freighter
211,D28,D228,Fairchild Dornier 228
351,PR1,PRM1,Hawker 390 Premier 1/1A
358,SFB,SF34,Saab 340B
198,CV2,CVLP,Convair 240 Passenger
90,74V,B74R,Boeing 747SR Freighter
345,PA1,*,Piper (Light aircraft-single piston engine)
45,722,B722,Boeing 727-200 Passenger
164,BH2,*,Bell (Helicopters)
150,AWZ,*,Augusta Westland 200


In [9]:
# Verification des NaN
display_nan(aircraft)

Synthèse :


{'aircraft_icao': 2, 'aircraft_name': 2}

,aircraft_iata,aircraft_icao,aircraft_name
77,748,NaN,None
78,748,NaN,None


Nb de NaN dans la colonne "aircraft_icao" : 2


,aircraft_iata,aircraft_icao,aircraft_name
77,748,NaN,None
78,748,NaN,None


Nb de NaN dans la colonne "aircraft_name" : 2


,aircraft_iata,aircraft_icao,aircraft_name
77,748,NaN,None
78,748,NaN,None


In [10]:
aircraft = aircraft.dropna()
aircraft = aircraft.reset_index(drop=True)

In [11]:
# on regarde si la colonne aircraft_iata ne comporte que des valeurs uniques
iata = aircraft['aircraft_iata'].tolist()
iata_uniques = list(set(iata))
print(len(iata))
print(len(iata_uniques))

380
379


In [12]:
for iata in iata_uniques:
    df = aircraft[aircraft['aircraft_iata'] == iata]
    if len(df) > 1:
        display(df)

,aircraft_iata,aircraft_icao,aircraft_name
97,762,B762,Boeing 767-200 Passenger
98,762,B762,Boeing 767-200 Passenger


In [13]:
# on supprime les rows dupliquées
aircraft = aircraft.drop_duplicates()
aircraft = aircraft.reset_index(drop=True)
iata = aircraft['aircraft_iata'].tolist()
iata_uniques = list(set(iata))
print(len(iata))
print(len(iata_uniques))

379
379


In [14]:
# on ajoute les données scrappées sur wikipedia pour ajouter un lien vers la page wiki
with open(f"{path_folder_data_wiki}/aircrafts_wiki.json", 'r', encoding='utf-8') as f:
    aircrafts_wiki = json.load(f)
aircraft_wiki = pd.DataFrame(aircrafts_wiki)
aircraft_wiki

,ICAO,IATA,MODEL,Link
0,A124,A4F,Antonov An-124 Ruslan,/wiki/Antonov_An-124_Ruslan
1,A140,A40,Antonov An-140,/wiki/Antonov_An-140
2,A148,A81,Antonov An-148,/wiki/Antonov_An-148
3,A158,A58,Antonov An-158,/wiki/Antonov_An-148#Variants
4,A19N,31N,Airbus A319neo,/wiki/Airbus_A319neo
...,...,...,...,...
325,WW24,WWP,Israel Aircraft Industries 1124 Westwind,/wiki/IAI_Westwind
326,Y12,YN2,Harbin Y-12,/wiki/Harbin_Y-12
327,YK40,YK4,Yakovlev Yak-40,/wiki/Yakovlev_Yak-40
328,YK42,YK2,Yakovlev Yak-42,/wiki/Yakovlev_Yak-42


In [15]:
# on vérifie la présence de valeurs uniques pour le code iata
aircraft_wiki = aircraft_wiki.dropna(subset=["IATA"])
aircraft_wiki = aircraft_wiki[aircraft_wiki['IATA'].isin(iata_uniques)]
iata_wiki = aircraft_wiki['IATA'].tolist()
iata_wiki_uniques = list(set(iata_wiki))
print(len(iata_wiki))
print(len(iata_wiki_uniques))

253
233


In [16]:
# on traite les valeurs multiples du code iata :
# pour les codes multtiples, on recherche la correspondance du code icao avec notre df aircraft
# si pas de correpondance, on supprime la row du df wiki
index_del = []
for iata in iata_wiki_uniques:
    df = aircraft_wiki[aircraft_wiki['IATA'] == iata]
    if len(df) > 1:
        df_index = list(df.index)
        df_air = aircraft[aircraft['aircraft_iata'] == iata]
        code_icao = df_air['aircraft_icao'].tolist()[0]
        if code_icao != "*":
            for i in df_index:
                if df.loc[i, 'ICAO'] != code_icao:
                    index_del.append(i)
        else:
            index_del.extend(df_index)
aircraft_wiki = aircraft_wiki.drop(index_del, axis=0)

In [17]:
# on merge les 2 df
aircraft_wiki.columns = ['icao', 'aircraft_iata', 'model', 'aircraft_link']
aircraft_wiki = aircraft_wiki[['aircraft_iata', 'aircraft_link']]
aircraft = pd.merge(aircraft, aircraft_wiki, how='left')
display(aircraft)

,aircraft_iata,aircraft_icao,aircraft_name,aircraft_link
0,100,F100,Fokker 100,/wiki/Fokker_100
1,141,B461,BAE Systems 146-100 Passenger,/wiki/British_Aerospace_146
2,142,B462,BAE Systems 146-200 Passenger,/wiki/British_Aerospace_146
3,143,B463,BAE Systems 146-300 Passenger,/wiki/British_Aerospace_146
4,14X,B461,BAE Systems 146-100 Freighter,NaN
...,...,...,...,...
374,YK2,YK42,Yakovlev Yak-42 / Yak-142,/wiki/Yakovlev_Yak-42
375,YK4,YK40,Yakovlev Yak-40,/wiki/Yakovlev_Yak-40
376,YN2,Y12,Harbin Yunshuji Y12,/wiki/Harbin_Y-12
377,YN7,AN24,Xian Yunshuji Y7,NaN


## Traitement du df airline

In [18]:
# affichage du df
airline = all_df[1]
print(airline.shape)
display(airline.sample(10))

(1136, 3)


,airline_iata,airline_icao,airline_name
1122,ZO,NaN,Zoom Air
985,VC,NaN,Viaair
906,T5,TUA,Turkmenistan Airlines
391,ER,SEP,Serene Air
506,HR,HHN,Hahn Air
33,2T,TUX,Tulpar Air Service
618,KY,KNA,Kunming Airlines
41,3C,CVA,Air Chathams
374,EA,EAL,European Air Express Luftverkehrsgesellschaft ...
90,4Y,OCN,Eurowings Discover


In [19]:
# Vérification des nan
display_nan(airline)

Synthèse :


{'airline_icao': 125}

,airline_iata,airline_icao,airline_name
14,1X,NaN,Branson Airexpress
15,2A,NaN,Deutsche Bahn AG
17,2C,NaN,Sncf
19,2E,NaN,Smokey Bay Air
28,2O,NaN,Island Air Service
...,...,...,...
1128,ZV,NaN,V Air Corporation
1132,ZZ,NaN,Alc
1133,ZZ,NaN,Alc
1134,ZZ,NaN,Alc


Nb de NaN dans la colonne "airline_icao" : 125


,airline_iata,airline_icao,airline_name
14,1X,NaN,Branson Airexpress
15,2A,NaN,Deutsche Bahn AG
17,2C,NaN,Sncf
19,2E,NaN,Smokey Bay Air
28,2O,NaN,Island Air Service
...,...,...,...
1128,ZV,NaN,V Air Corporation
1132,ZZ,NaN,Alc
1133,ZZ,NaN,Alc
1134,ZZ,NaN,Alc


In [20]:
# On supprime les 3 dernières lignes 
airline = airline.drop([1133, 1134, 1135], axis=0)

# on regarde si la colonne airline_data ne comporte que des valeurs uniques
print(airline.shape)
uniques_iata = airline['airline_iata'].unique().tolist()
print(len(uniques_iata))

(1133, 3)
1127


In [21]:
# on regarde les rows avec un code iata multiple
count = pd.Series(airline['airline_iata'].tolist()).value_counts()
codes_iata = []
for i in range(len(count)):
    if count[i]>1:
        codes_iata.append(count.index[i])
        
display(airline[airline['airline_iata'].isin(codes_iata)])

,airline_iata,airline_icao,airline_name
88,4Y,BGA,Airbus Transport International
89,4Y,BGA,Airbus Transport International
90,4Y,OCN,Eurowings Discover
91,4Y,BGA,Airbus Transport International
92,4Y,BGA,Airbus Transport International
101,5H,FFV,Fly540
102,5H,FFV,Fly540
635,LH,DLH,Lufthansa
636,LH,GEC,Lufthansa Cargo


In [22]:
# On supprime les lignes dupliquées
airline = airline.drop([89, 90, 91, 92, 102, 636], axis=0)
airline = airline.reset_index(drop = True)
print(airline.shape)

(1127, 3)


## Traitement du df airport

In [23]:
airport = all_df[2]
print(airport.shape)
display(airport.sample(10))

(11681, 9)


,airport_iata,airport_city_code,airport_country_code,airport_location_type,airport_utc_offset,airport_timezone_id,airport_name,airport_latitude,airport_longitude
3729,ISN,ISN,US,Airport,-06:00,America/Chicago,Williston,48.1792,-103.6433
5084,LUZ,LUZ,PL,Airport,+01:00,Europe/Warsaw,Lublin,51.2314,22.6906
7370,QPE,QPE,BR,Off-Line Point,-03:00,America/Araguaina,Petropolis Off-Line Pt,-22.5167,-43.1667
6088,NQZ,NQZ,KZ,Airport,+06:00,Asia/Almaty,Nur-Sultan (Astana),51.0231,71.4669
3154,HAU,HAU,NO,Airport,+01:00,Europe/Oslo,Haugesund,59.3453,5.2083
8192,SIE,SIE,PT,Airport,+00:00,Europe/Lisbon,Sines,37.9500,-8.8667
8072,SDN,SDN,NO,Airport,+01:00,Europe/Oslo,Sandane Anda,61.8300,6.1075
10215,XLR,XLR,FR,Off-Line Point,+01:00,Europe/Paris,Libourne Off-Line Pt,44.9167,-0.2333
8760,TEN,TEN,CN,Airport,+08:00,Asia/Shanghai,Tongren Fenhuang,27.8833,109.3083
6344,OLK,OLK,PY,Airport,-04:00,America/Asuncion,Fuerte Olimpo,-21.0458,-57.8828


In [24]:
# location_type : on se limite aux airports
airport = airport[airport['airport_location_type'] == "Airport"]

# on supprime la colonne 'airport_location_type'
airport = airport.drop(columns=['airport_location_type'])

airport = airport.reset_index(drop=True)
airport.shape

(9728, 8)

In [25]:
iata_code = airport["airport_iata"].tolist()
print(len(iata_code))
iata_code_uniques = (list(set(iata_code)))
print(len(iata_code_uniques))

9728
9723


In [26]:
# on regarde les rows avec un code iata multiple
count = pd.Series(airport['airport_iata'].tolist()).value_counts()
codes_iata = []
for i in range(len(count)):
    if count[i]>1:
        codes_iata.append(count.index[i])
        
display(airport[airport['airport_iata'].isin(codes_iata)])

,airport_iata,airport_city_code,airport_country_code,airport_utc_offset,airport_timezone_id,airport_name,airport_latitude,airport_longitude
97,ADX,ADX,GB,+00:00,Europe/London,St Andrews Raf Leuchars,56.3667,-2.8667
98,ADX,ADX,GB,+00:00,Europe/London,St Andrews Raf Leuchars,56.3667,-2.8667
2840,GOM,GOM,CD,+02:00,Africa/Lubumbashi,None,-1.6708,29.2383
2841,GOM,GOM,CD,+02:00,Africa/Lubumbashi,Goma,-1.6708,29.2383
2861,GPS,GPS,EC,-06:00,Pacific/Galapagos,None,-0.4539,-90.2658
2862,GPS,GPS,EC,-06:00,Pacific/Galapagos,Baltra Island Seymour,-0.4539,-90.2658
3046,HBX,HBX,IN,+05:30,Asia/Kolkata,Hubli,15.3617,75.0869
3047,HBX,HBX,IN,+05:30,Asia/Kolkata,None,15.3617,75.0869
3068,HDN,HDN,US,-07:00,America/Denver,Hayden Yampa Valley,40.4833,-107.2236
3069,HDN,HDN,US,-07:00,America/Denver,None,40.4833,-107.2236


In [27]:
# on supprime les rows dupliquées
airport = airport.drop([98, 2840, 2861, 3047, 3069], axis=0)
airport = airport.reset_index(drop = True)
print(airport.shape)

(9723, 8)


In [28]:
display_nan(airport)

Synthèse :


{}

,airport_iata,airport_city_code,airport_country_code,airport_utc_offset,airport_timezone_id,airport_name,airport_latitude,airport_longitude


In [29]:
# on utilise les données airport scrappées sur wikipedia pour ajouter le code ICAO et un lien
with open(f"{path_folder_data_wiki}/airports_wiki.json", "r", encoding='utf-8') as f:
    airports_wiki = json.load(f)
airports_wiki[:3]

[{'IATA': 'AAA',
  'ICAO': 'NTGA',
  'AirportName': 'Anaa Airport',
  'AirportLink': '/wiki/Anaa_Airport',
  'LocationServed': 'Anaa, Tuamotus, French Polynesia'},
 {'IATA': 'AAB',
  'ICAO': 'YARY',
  'AirportName': 'Arrabury Airport',
  'AirportLink': '/wiki/Arrabury_Airport',
  'LocationServed': 'Arrabury, Queensland, Australia'},
 {'IATA': 'AAC',
  'ICAO': 'HEAR',
  'AirportName': 'El Arish International Airport',
  'AirportLink': '/wiki/El_Arish_International_Airport',
  'LocationServed': 'El Arish, Egypt'}]

In [30]:
df_wiki = pd.DataFrame(airports_wiki)
df_wiki = df_wiki[['IATA', 'ICAO', 'AirportLink']]
df_wiki.columns = ['airport_iata', 'airport_icao', 'airport_link']
df_wiki

,airport_iata,airport_icao,airport_link
0,AAA,NTGA,/wiki/Anaa_Airport
1,AAB,YARY,/wiki/Arrabury_Airport
2,AAC,HEAR,/wiki/El_Arish_International_Airport
3,AAD,HCAD,/wiki/Adado_Airport
4,AAE,DABB,/wiki/Rabah_Bitat_Airport
...,...,...,...
9075,ZYL,VGSY,/wiki/Osmani_International_Airport
9076,ZZE,UBBZ,/wiki/Zangilan_International_Airport
9077,ZZO,UHSO,/wiki/Zonalnoye_Airport
9078,ZZU,FWUU,/wiki/Mzuzu_Airport


In [31]:
airport = pd.merge(airport, df_wiki, how='left')
airport

,airport_iata,airport_city_code,airport_country_code,airport_utc_offset,airport_timezone_id,airport_name,airport_latitude,airport_longitude,airport_icao,airport_link
0,AAA,AAA,PF,-10:00,Pacific/Tahiti,Anaa,-17.3525,-145.5100,NTGA,/wiki/Anaa_Airport
1,AAB,AAB,AU,+10:00,Australia/Brisbane,Arrabury,-26.6911,141.0472,YARY,/wiki/Arrabury_Airport
2,AAC,AAC,EG,+02:00,Africa/Cairo,El Arish International,31.0733,33.8358,HEAR,/wiki/El_Arish_International_Airport
3,AAD,AAD,SO,+03:00,Africa/Mogadishu,Adado,6.0961,46.6375,HCAD,/wiki/Adado_Airport
4,AAE,AAE,DZ,+01:00,Africa/Algiers,Annaba Rabah Bitat,36.8222,7.8092,DABB,/wiki/Rabah_Bitat_Airport
...,...,...,...,...,...,...,...,...,...,...
9718,ZZE,ZZE,AZ,+04:00,Asia/Baku,Zangilan,39.0989,46.7344,UBBZ,/wiki/Zangilan_International_Airport
9719,ZZO,ZZO,RU,+11:00,Asia/Srednekolymsk,Zonalnoye,50.6689,142.7611,UHSO,/wiki/Zonalnoye_Airport
9720,ZZU,ZZU,MW,+02:00,Africa/Blantyre,Mzuzu,-11.4447,34.0117,FWUU,/wiki/Mzuzu_Airport
9721,ZZV,ZZV,US,-05:00,America/New_York,Zanesville Municipal,39.9500,-81.8833,KZZV,/wiki/Zanesville_Municipal_Airport


## Traitement du df country

In [32]:
# On regarde les données de Lufthansa pour country
country = all_df[4]
country

,country_code,country_name
0,AD,Andorra
1,AE,United Arab Emirates
2,AF,Afghanistan
3,AG,"Antigua And Barbuda, Leeward Islands"
4,AI,"Anguilla, Leeward Islands"
...,...,...
234,YE,Yemen
235,YT,Mayotte
236,ZA,South Africa
237,ZM,Zambia


In [33]:
display_nan(country)

Synthèse :


{}

,country_code,country_name


In [34]:
# on verifie que code_pays sont uniques
codes_pays = country['country_code']
print(len(codes_pays))
codes_pays_uniques = list(set(codes_pays))
print(len(codes_pays_uniques))

239
238


In [35]:
# on regarde les rows avec un code pays multiple
count = pd.Series(country['country_code'].tolist()).value_counts()
codes_pays_ = []
for i in range(len(count)):
    if count[i]>1:
        codes_pays_.append(count.index[i])
        
display(country[country['country_code'].isin(codes_pays_)])

,country_code,country_name
99,IR,Iran Islamic Republic Of
100,IR,Iran Islamic Republic Of


In [36]:
# on supprime les rows dupliquées
country = country.drop([100], axis=0)
country = country.reset_index(drop = True)
print(country.shape)

(238, 2)


In [37]:
# on verifie que tous les codes country de airport sont présents
codes_pays_airports = airport['airport_country_code'].unique().tolist()
missing = list(set(codes_pays_airports) - set(codes_pays))
print(missing)

[]


In [38]:
# on utilise les données scrappées sur wikipedia pour rajouter des infos

In [39]:
with open(f"{path_folder_data_wiki}/country_wiki.json", "r", encoding='utf-8') as f:
    country_wiki = json.load(f)
df_wiki = pd.DataFrame(country_wiki)
df_wiki.head()

,CountryName,CountryLink,CountryFlag,Alpha2Code,Alpha3Code,NumericCode
0,Afghanistan,/wiki/Afghanistan,//upload.wikimedia.org/wikipedia/commons/thumb...,AF,AFG,004
1,Åland Islands,/wiki/%C3%85land_Islands,//upload.wikimedia.org/wikipedia/commons/thumb...,AX,ALA,248
2,Albania,/wiki/Albania,//upload.wikimedia.org/wikipedia/commons/thumb...,AL,ALB,008
3,Algeria,/wiki/Algeria,//upload.wikimedia.org/wikipedia/commons/thumb...,DZ,DZA,012
4,American Samoa,/wiki/American_Samoa,//upload.wikimedia.org/wikipedia/commons/thumb...,AS,ASM,016


In [40]:
df_wiki = df_wiki[['Alpha2Code', 'Alpha3Code', 'NumericCode', 'CountryFlag', 'CountryLink']]
df_wiki.columns = ['country_code', 'country_iso3', 'country_num', 'country_flag', 'country_link']

In [41]:
country = pd.merge(country, df_wiki, how='left')
country

,country_code,country_name,country_iso3,country_num,country_flag,country_link
0,AD,Andorra,AND,020,//upload.wikimedia.org/wikipedia/commons/thumb...,/wiki/Andorra
1,AE,United Arab Emirates,ARE,784,//upload.wikimedia.org/wikipedia/commons/thumb...,/wiki/United_Arab_Emirates
2,AF,Afghanistan,AFG,004,//upload.wikimedia.org/wikipedia/commons/thumb...,/wiki/Afghanistan
3,AG,"Antigua And Barbuda, Leeward Islands",ATG,028,//upload.wikimedia.org/wikipedia/commons/thumb...,/wiki/Antigua_and_Barbuda
4,AI,"Anguilla, Leeward Islands",AIA,660,//upload.wikimedia.org/wikipedia/commons/thumb...,/wiki/Anguilla
...,...,...,...,...,...,...
233,YE,Yemen,YEM,887,//upload.wikimedia.org/wikipedia/commons/thumb...,/wiki/Yemen
234,YT,Mayotte,MYT,175,//upload.wikimedia.org/wikipedia/en/thumb/c/c3...,/wiki/Mayotte
235,ZA,South Africa,ZAF,710,//upload.wikimedia.org/wikipedia/commons/thumb...,/wiki/South_Africa
236,ZM,Zambia,ZMB,894,//upload.wikimedia.org/wikipedia/commons/thumb...,/wiki/Zambia


## Traitement du df city

In [42]:
## Intérêt de cette table ??
# La seule info supplémentaire est le nom de la ville  => intérêt à l'ajouter directement dans la table airport ??

In [43]:
city

,city_code,city_country_code,city_utc_offset,city_timezone_id,city_name,city_airport_code
0,AAA,PF,-10:00,Pacific/Tahiti,Anaa,AAA
1,AAB,AU,+10:00,Australia/Brisbane,Arrabury,AAB
2,AAC,EG,+02:00,Africa/Cairo,El Arish,AAC
3,AAD,SO,+03:00,Africa/Mogadishu,Adado,AAD
4,AAE,DZ,+01:00,Africa/Algiers,Annaba,AAE
...,...,...,...,...,...,...
10695,ZZL,DZ,+01:00,Africa/Algiers,L'arbaa Nait Irathen,ZZL
10696,ZZO,RU,+11:00,Asia/Srednekolymsk,Zonalnoye,ZZO
10697,ZZP,RS,+01:00,Europe/Belgrade,Pozarevac,ZZP
10698,ZZQ,CN,+08:00,Asia/Shanghai,Cheung Sha Wan,ZZQ


In [44]:
display_nan(city)

Synthèse :


{'city_airport_code': 104}

,city_code,city_country_code,city_utc_offset,city_timezone_id,city_name,city_airport_code
21,AAW,PK,+05:00,Asia/Karachi,Abbottabad,NaN
66,ACQ,ES,+01:00,Europe/Madrid,Estacion De Alicante,NaN
191,AIY,US,-05:00,America/New_York,Atlantic City,NaN
279,ANA,KE,+03:00,Africa/Nairobi,Masai Mara,NaN
433,AUB,BR,-04:00,America/Cuiaba,Itauba,NaN
...,...,...,...,...,...,...
9396,XGL,FR,+01:00,Europe/Paris,Angouleme,NaN
9562,XPI,FR,+01:00,Europe/Paris,Gare Perpignan Sncf,NaN
9652,XTZ,FI,+02:00,Europe/Helsinki,Turku,NaN
9697,XWA,US,-06:00,America/Chicago,Williston,NaN


Nb de NaN dans la colonne "city_airport_code" : 104


,city_code,city_country_code,city_utc_offset,city_timezone_id,city_name,city_airport_code
21,AAW,PK,+05:00,Asia/Karachi,Abbottabad,NaN
66,ACQ,ES,+01:00,Europe/Madrid,Estacion De Alicante,NaN
191,AIY,US,-05:00,America/New_York,Atlantic City,NaN
279,ANA,KE,+03:00,Africa/Nairobi,Masai Mara,NaN
433,AUB,BR,-04:00,America/Cuiaba,Itauba,NaN
...,...,...,...,...,...,...
9396,XGL,FR,+01:00,Europe/Paris,Angouleme,NaN
9562,XPI,FR,+01:00,Europe/Paris,Gare Perpignan Sncf,NaN
9652,XTZ,FI,+02:00,Europe/Helsinki,Turku,NaN
9697,XWA,US,-06:00,America/Chicago,Williston,NaN


In [45]:
# on verifie que code_city sont uniques
codes_city = city['city_code']
print(len(codes_city))
codes_city_uniques = list(set(codes_city))
print(len(codes_city_uniques))

10700
10693


In [46]:
# on regarde les rows avec un code pays multiple
count = pd.Series(city['city_code'].tolist()).value_counts()
codes_city_ = []
for i in range(len(count)):
    if count[i]>1:
        codes_city_.append(count.index[i])
        
display(city[city['city_code'].isin(codes_city_)])

,city_code,city_country_code,city_utc_offset,city_timezone_id,city_name,city_airport_code
99,AEB,CN,+08:00,Asia/Shanghai,Baise,AEB
100,AEB,CN,+08:00,Asia/Shanghai,Baise,AEB
2536,GBI,IN,+05:30,Asia/Kolkata,Gulbarga,GBI
2537,GBI,IN,+05:30,Asia/Kolkata,Gulbarga,GBI
3491,IXX,IN,+05:30,Asia/Kolkata,Bidar Airport,IXX
3492,IXX,IN,+05:30,Asia/Kolkata,Bidar,IXX
3493,IXX,IN,+05:30,Asia/Kolkata,Bidar,IXX
9026,WAS,US,-05:00,America/New_York,Washington,"[BOF, BWI, BZS, DCA, GBO, IAD, JPN, MTN, ZBP, ..."
9027,WAS,US,-05:00,America/New_York,Washington,"[BOF, BWI, BZS, DCA, GBO, IAD, JPN, MTN, ZBP, ..."
9028,WAS,US,-05:00,America/New_York,Washington,"[BOF, BWI, BZS, DCA, GBO, IAD, JPN, MTN, ZBP, ..."


In [47]:
# on supprime les rows dupliquées
city = city.drop([100, 2537, 3492, 3493, 9027, 9028, 10223], axis=0)
city = city.reset_index(drop = True)
print(city.shape)

(10693, 6)


In [48]:
# on verifie que tous les codes city de airport sont présents
codes_city_airports = airport['airport_city_code'].unique().tolist()
missing = list(set(codes_city_airports) - set(codes_city))
print(missing)

['ZZV']


In [49]:
city_ = city.dropna(subset=['city_airport_code'])
city_ = city_.reset_index(drop=True)
print(len(city_))
print(len(city_['city_code'].unique().tolist()))
city_dic = {}
for i in range(len(city_)):
    codes = city_.loc[i, 'city_airport_code']
    city_code = city_.loc[i, 'city_code']
    if codes[0] == '[':
        codes_array = codes[2:-2].split("', '")
        city_dic[city_code] = codes_array
    else:
        city_dic[city_code] = codes

        

10589
10589


In [50]:
city_dic

{'AAA': 'AAA',
 'AAB': 'AAB',
 'AAC': 'AAC',
 'AAD': 'AAD',
 'AAE': 'AAE',
 'AAF': 'AAF',
 'AAG': 'AAG',
 'AAH': ['AAH', 'AAW', 'XHJ'],
 'AAI': 'AAI',
 'AAJ': 'AAJ',
 'AAK': 'AAK',
 'AAL': ['AAL', 'ZTG'],
 'AAM': 'AAM',
 'AAN': ['AAN', 'ZVH'],
 'AAO': 'AAO',
 'AAQ': 'AAQ',
 'AAR': ['AAR', 'ZBU', 'ZID', 'ZJH'],
 'AAS': 'AAS',
 'AAT': 'AAT',
 'AAU': 'AAU',
 'AAV': 'AAV',
 'AAX': 'AAX',
 'AAY': 'AAY',
 'AAZ': 'AAZ',
 'ABA': 'ABA',
 'ABB': 'ABB',
 'ABC': ['ABC', 'EEM', 'XJJ'],
 'ABD': 'ABD',
 'ABE': 'ABE',
 'ABF': 'ABF',
 'ABG': 'ABG',
 'ABH': 'ABH',
 'ABI': ['ABI', 'DYS'],
 'ABJ': 'ABJ',
 'ABK': 'ABK',
 'ABL': 'ABL',
 'ABM': 'ABM',
 'ABN': 'ABN',
 'ABO': 'ABO',
 'ABP': 'ABP',
 'ABQ': 'ABQ',
 'ABR': 'ABR',
 'ABS': 'ABS',
 'ABT': 'ABT',
 'ABU': 'ABU',
 'ABV': 'ABV',
 'ABW': 'ABW',
 'ABX': 'ABX',
 'ABY': ['ABY', 'NAB'],
 'ABZ': ['ABZ', 'ZXA'],
 'ACA': 'ACA',
 'ACB': 'ACB',
 'ACC': 'ACC',
 'ACD': 'ACD',
 'ACE': 'ACE',
 'ACF': 'ACF',
 'ACH': 'ACH',
 'ACI': 'ACI',
 'ACJ': 'ACJ',
 'ACK': 'ACK',


In [51]:
display(city[city['city_code'] == 'WAS'])

,city_code,city_country_code,city_utc_offset,city_timezone_id,city_name,city_airport_code
9022,WAS,US,-05:00,America/New_York,Washington,"[BOF, BWI, BZS, DCA, GBO, IAD, JPN, MTN, ZBP, ..."


In [52]:
airport[airport['airport_city_code'] == 'WAS']

,airport_iata,airport_city_code,airport_country_code,airport_utc_offset,airport_timezone_id,airport_name,airport_latitude,airport_longitude,airport_icao,airport_link
909,BOF,WAS,US,-05:00,America/New_York,Washington Bolling Afb Heliport,38.9167,-77.0167,NaN,NaN
1093,BWI,WAS,US,-05:00,America/New_York,Baltimore Washington,39.1753,-76.6683,KBWI,/wiki/Baltimore%E2%80%93Washington_Internation...
1792,DCA,WAS,US,-05:00,America/New_York,Washington Ronald Reagan National,38.8522,-77.0378,KDCA,/wiki/Ronald_Reagan_Washington_National_Airport
3352,IAD,WAS,US,-05:00,America/New_York,Washington Dulles,38.9444,-77.4558,KIAD,/wiki/Washington_Dulles_International_Airport
3812,JPN,WAS,US,-05:00,America/New_York,Washington Pentagon Ahp,38.8500,-77.0333,NaN,NaN
5422,MTN,WAS,US,-05:00,America/New_York,Baltimore Martin State,39.3261,-76.4139,KMTN,/wiki/Martin_State_Airport


In [53]:
airport[airport['airport_iata'] == 'BOF']

,airport_iata,airport_city_code,airport_country_code,airport_utc_offset,airport_timezone_id,airport_name,airport_latitude,airport_longitude,airport_icao,airport_link
909,BOF,WAS,US,-05:00,America/New_York,Washington Bolling Afb Heliport,38.9167,-77.0167,NaN,NaN


In [54]:
print(airport.columns)
print(airline.columns)
print(aircraft.columns)
print(country.columns)
print(city.columns)

Index(['airport_iata', 'airport_city_code', 'airport_country_code',
       'airport_utc_offset', 'airport_timezone_id', 'airport_name',
       'airport_latitude', 'airport_longitude', 'airport_icao',
       'airport_link'],
      dtype='object')
Index(['airline_iata', 'airline_icao', 'airline_name'], dtype='object')
Index(['aircraft_iata', 'aircraft_icao', 'aircraft_name', 'aircraft_link'], dtype='object')
Index(['country_code', 'country_name', 'country_iso3', 'country_num',
       'country_flag', 'country_link'],
      dtype='object')
Index(['city_code', 'city_country_code', 'city_utc_offset', 'city_timezone_id',
       'city_name', 'city_airport_code'],
      dtype='object')
